# Setting the Environment

## Installing

In [ ]:
# !pip3 install pythainlp
# !pip3 install https://github.com/PyThaiNLP/thai_sentiment_analysis/archive/master.zip
# !pip3 install kenlm
# !pip3 install pypdf
# !pip3 install pytesseract
# !pip3 install PyMuPDF
# !pip3 install transformers
# !pip3 install thai_sentiment
# !pip3 install scikit-learn
# !pip3 install sklearn_crfsuite
# !pip3 install emoji
# !pip3 install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
# !pip3 install matplotlib_venn
# !pip3 install torch

## Importing

In [ ]:
from pythainlp.corpus.common import thai_stopwords
from wordcloud import WordCloud, STOPWORDS
from pythainlp import word_tokenize
from pythainlp.ulmfit import *
from pythaisa import *
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [ ]:
sample1 = pd.read_csv(r'datasets\sample.csv', sep='\t', names=['text', 'sentiment'], header=None)
sample2 = pd.read_csv(r'datasets\wisesight_3.csv', names=['text', 'sentiment'], header=None, skiprows=1)

# data = pd.read_csv(r'datasets\test.csv', encoding='utf8', header=None)

data = pd.read_excel(r'datasets\คอมมูเกม.xlsx', header=None)
# data = pd.read_excel(r'datasets\คอมมูขายของออนไลน์.xlsx', header=None)
# data = pd.read_excel(r'datasets\คอมมูคุยเรื่อง 18+.xlsx', header=None)

## Default parameters

In [ ]:
# API parameters

API_KEY = 'kHIllIH4ODKsOvvi7QJINN5FIzf6sFgR'
API_FOR_THAI = "https://api.aiforthai.in.th"
SSSENSE_ENDPOINT = f"{API_FOR_THAI}/ssense"
TEXT_CLEANSING_ENDPOINT = f"{API_FOR_THAI}/textcleansing"

HEADERS = {"apikey": API_KEY}

# Processing

## Model 1

In [ ]:
thai_stopwords = list(thai_stopwords())

def text_process(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ"))
    final = word_tokenize(final)
    final = " ".join(word for word in final)
    final = " ".join(word for word in final.split() if word.lower not in thai_stopwords)
    return final

def model1(df):
    sample = df.replace(np.nan, '', regex=True)

    sample['text_tokens'] = sample['text'].apply(text_process)

    x = sample[['text_tokens']]
    y = sample['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=101)

    global cvec
    cvec = CountVectorizer(analyzer=lambda x:x.split(' '))
    cvec.fit_transform(X_train['text_tokens'])
    print(cvec.vocabulary_)

    train_bow = cvec.transform(X_train['text_tokens'])
    pd.DataFrame(train_bow.toarray(), columns=cvec.get_feature_names_out(), index=X_train['text_tokens'])

    global lr
    lr = LogisticRegression(max_iter=20000)
    lr.fit(train_bow, y_train)

    test_bow = cvec.transform(X_test['text_tokens'])
    test_predictions = lr.predict(test_bow)
    print(classification_report(test_predictions, y_test))

model1(sample2)

- wisesight_1

positive

| sentiment | precision | recall | f1   |
|-----------|-----------|--------|------|
| neg       | 0.70      | 0.74   | 0.72 |
| neu       | 0.57      | 0.57   | 0.57 |
| pos       | 0.65      | 0.61   | 0.63 |

- wisesight_2

| sentiment | precision | recall | f1   |
|-----------|-----------|--------|------|
| neg       | 0.74      | 0.77   | 0.76 |
| neu       | 0.73      | 0.66   | 0.69 |
| pos       | 0.55      | 0.62   | 0.59 |

- wisesight_3

negative

| sentiment | precision | recall | f1   |
|-----------|-----------|--------|------|
| neg       | 0.77      | 0.81   | 0.79 |
| neu       | 0.62      | 0.59   | 0.60 |
| pos       | 0.62      | 0.61   | 0.62 |

In [ ]:
# model1

ml1_data = data

lst = ml1_data.iloc[:, 0].tolist()

def sentiment(lst):
    text = []
    prediction = []
    for item in lst:
        my_tokens = text_process(item)
        my_bow = cvec.transform(pd.Series([my_tokens]))
        my_predictions = lr.predict(my_bow)
        text.append(item)
        prediction.append(my_predictions[0])
    return list(zip(text, prediction))

sentiment_lst_ml1 = sentiment(lst)
print(sentiment_lst_ml1)

- คอมมูเกม
[('ดอทเอ เกมเหี้ย ควายๆ ขยะ กุจะไม่เล่นอีก😡', 'neg'), ('แค่ดูก็พอครับ ดอทเอยุคนี้มันหมา', 'neu'), ('ในวันที่เหนื่อย ในวันที่ท้อ อยากเปิดดอทเอขึ้นมาเล่นแล้วด่าคน', 'pos'), ('ดอทเอ คือเซฟโซน ถึงพวกโง่สวะขยะขี้ตีนจะเยอะ แล้วต้องประสาทแดกกับมันแค่ไหน แต่ก็สนุกที่ได้ด่าพวกส้นตีน', 'neg'), ('เหมือนดอทเออะอีเหี้ย เอา 10k+ ชั่วโมงกูคืนมาเย๋ดแม่กาเบ็นกาเบจ 5555555555555', 'neg'), ('น้องเค่อฉิงสวยขึ้นทุกปี😭😭😭😭', 'pos'), ('มิโฮโยอย่าทำแบบนี้ อย่าดึงกุกลับไป ทั้งรางดาวทั้งเกนชินเลยไอเหี้ยเอ่ย กุอุส่าเลิกเล่นเกมเด็ดขาดมาแล้วแทะๆ', 'neg'), ('รางดาวมันดูไปได้ไกลกว่าเกนชินแบบมากๆๆๆๆๆ เซตติ้งเป็นจักรวาล มีดาวใหม่ๆได้เรื่อยๆไม่กระจุกอยู่ไม่กี่ประเทด ตัวละครสามารถมีร่างใหม่พาธใหม่ออกมาขายได้อีก รักฮงไกจนแบบ เกนชินแม่งก็แค่ฟลุกดังจนเป็นตัวทำเงินเอาไปทำเกมอื่นจริงนั่นแหละ55555555', 'neg'), ('70โรล ออกพี่ติ๊ก อีเหี้ยยยยยย หลุดทุกตู้เลยวะ ไม่ต้องรักกุขนาดนี้ก็ได้มั้ง เกมหมา เบื่อ กุเก็บเจมตั้งหลายเดือน ไอ้เหี้ยๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ', 'neg'), ('เกมหมาเหวี่ยงคนเล่นดีๆมาเจอกุหน่อยได้ปะ เจอแต่hereไรไม่รู้เนี่ย', 'neg'), ('รีบเข้าเกมหมาเพื่อดูตัวใหม่', 'neu'), ('เกมหมาอิสัสกุจะชนะก็แพ้ค้างโคตรพ่อโคตรแม่สรุปชดเชยกุเพชรแดงปะแล้วกุต้องมานั่งเล่นแก้ดาวตัวเองอีกตั้งแต่เมื่อคืนละอะไรของมึง', 'neg'), ('แต่แมพถ้าจะเจอเควส ต้องเดินเข้าซอกเข้าซอนอะ ถ้าไม่ได้ขยันเดินคือไปไม่ถึงเควส 😔 แต่สนุก พัซเซิลสนุกเสมอเลยเกมหมานี่ ขอบจัยนะ', 'neg'), ('จริง เกมหมาแรงค์ล่างๆที่เล่นกันเหมือนลิงก็ผู้ชายทั้งนั้น แล้วพอโดนพูดเรื่องจริงใส่ก็โมโหโยนเกม', 'neg'), ('ยื่นเสียมให้มันค่ะ \n//ยินดีต้อนรับสู่เมวี่เกษตรtools(โทนเสียงเกมROV) \nเอาจริงเฉยๆ ค่ะ แซะมาถ้าดูแล้วฉลาดสมกับที่ควรลับฝีปากด้วยก็ด่า ถือว่าฝึกสกิล \nแต่หมาเห่า เราสามารถทำการไล่ได้ ไม่จำเป็นต้องไปเห่าภาษาเดียวกับหมาก็ได้ \nเข้าใจใช่ไหมคะคนเก่ง', 'neg'), ('พิสูจน์รักแท้ด่วยเกมหมา คือแอลกะโอเวอร์คุกเก็ต คือกุผ่านได้สบายมาก แต่แฟนกูจะหวนคืนสู่วานรทุกทีทุกครั้งที่เล่นโอเวอร์คุกเก็ตเลย กุงง กะแอลคือชิล มันเป็นอะไรนัก เกมออกจะสนุก 5554555555555', 'neg'), ('เรื่องนัทลานแฟนตาซีไปมั้ย คำตอบคือมันทำให้ผู้เล่น ’รู้สึก‘ แบบนั้นอะ คือทุกคนรู้อยู่แล้วว่าแต้แว้ดมันมีเทคโนโลยีพีคๆ มาทุกเมือง งั้นขอถามกลับว่า “ทำไมรอบนี้หมีทำให้ผู้เล่นรู้สึกไม่เอะใจแบบเมืองอื่นไม่ได้ล่ะ?” เมืองอื่นเรารู้สึกว่ามันปกติ แต่นัทลานมันโดดมากตั้งแต่แนวอาร์ต (+)', 'neg'), ('รู้สึกได้เลยว่าทีมรางดาวรักผบบ(mc)จริงๆ เพราะเราก็รักผบบมาก เวลาที่รักเราก็จะคอยสังเกตเค้า สังเกตรอบตัวเค้ามากกว่าตัวอื่นเป็นพิเศษ อะไรนิดหน่อยถ้ามันไม่ดีก็จะรู้สึกได้ ยอมรับว่ามีอคติเรื่องทรีตมาก่อนเพราะอีกเกม ในอนาคตถ้ามันทำไม่ดีก็จะด่า แต่ตอนนี้ใกล้จะ 2 ปีแล้ว ยังประทับใจอยู่', 'neg'), ('เอเป็กเกมไวมากกก ตอนเล่นเกม เอเป็ก / xdefiant / ow ไม่รู้สึกเวียนหัวนะ เพราะชินกับคตวามเร็วเกมที่มันหมุนไวๆ แบบว่าเห็นว่ามันช้าไปแล้ว แต่พอเล่นพับจีปุ้บเกมแม่งช้ามาก อินี่นอนยิงลุกยิงสลับมุมกล้อง ตาย อิดอก เกมอะไรกุเคยเล่นไปได้ยังไง 555555555', 'neg'), ('เล่นเอเป็ก สาทุ้บุนอย่าเจอคนจีนในตี้ แม่งแบบ เฮ้อ กุว่าใงแยกเซิฟแะดีละ \n', 'neg'), ('บ่นเอเป็ก \n\n98.99%จากการเล่นเอเป็กมา3ปี ผุ้เล่นคริปโตมึงจะห่วงเล่นโดรนยันพ่อมึงตายเลยมั้ย', 'neu'), ('ใครแนะนำไรไม่รู้ แต่อาโอวีอย่าเปิดช่องแชทอ่าน เกมหมา คนหมากว่า มีแต่พวกจู๋อุบาดปากเก่งแร้งได \n\n', 'neg'), ('มายกอตเรนโบว์ซิก ลงมือถือละ จะเปลี้ยนโทรสับใหม่ก้อเพราะจะเล่นเกมนี่เเหละ55555', 'neu'), ('เมอซี่ชุปทั้งทีมคือความฟิรของโอเวอร์วอชคลาสสิคจ้าแม่ \n', 'neg'), ('เล่นโอเวอร์วอชคลาสสิคแล้วสงสัยว่าตอนนั้นเขาเล่นกันได้ไงวะเนี่ย คือเรามาไม่ทันยุคนั้น 5555', 'neg')]

- คอมมูขายของ
[('พรีlongchamp รอสินค้า10-15วัน ราคานี้ขยายเวลารับถึงวันที่30คับ mini 2900 M 3900 L 4300 backpack 4200', 'neu'), ('📍 ราคาหลักสิบทุกตัวเลยน้า 📍 \n✿ มีเยอะกว่าในรูปมาก ๆ เลยค่ะ ✿ #ส่งต่อเสื้อผ้า #ส่งต่อเสื้อผ้ามือสอง #เสื้อผ้ามือสอง #เสื้อผ้ามือสองราคาถูก #ส่งต่อเสื้อผ้าราคาถูก #โล๊ะตู้เสื้อผ้า #ส่งต่อเสื้อผ้ามือ2 #ส่งต่อเสื้อผ้าy2k', 'neu'), ('pls kindly rt\n\npre-order today - 30 nov 18:00\nตะขออะคลิลิค carabiner 🍮🍒\n\n★ pudding ver. ชิ้นละ 160 บาท\n★ cherry ver. ชิ้นละ 160 บาท\nค่าส่งลงทะเบียน 30 บาททุกออเดอร์\n\nสนใจสั่งซื้อกรอกฟอร์มได้เลยค่ะ\nforms.gle/8JQMPV811gAzkN…', 'neu'), ('📢ปลายเดือนนี้ลดแรง\n𝙃𝙚𝙖𝙧𝙩 𝙋𝙚𝙧𝙘𝙚𝙣𝙩 จัดโปรลดลิปทั้งร้าน\nเริ่มต้น 𝟑𝟔𝟗.- เท่านั้น !!\n📍เฉพาะวันที่ 25 - 30 พ.ย. 67\n\n✨ซื้อครบ 400.- ลดเพิ่ม 30.-\n✨ซื้อครบ 500.- ลดเพิ่ม 60.-', 'neu'), ('คล้องไหล่หนังสีดำ สภาพ98% ฐ10 ส6 \n\n🌷169฿\n\n#ส่งต่อ #ส่งต่อกระเป๋า #ส่งต่อกระเป๋ามือสอง #ส่งต่อกระเป๋ามือ2 #กระเป๋ามือสอง #กระเป๋าเกาหลี #ส่งต่อstylist #กระเป๋าสะพาย #ส่งต่อกระเป๋าcarlyn #กระเป๋าgentlewoman', 'neu'), ('💥ราคาพิเศษสุดจึ้งใจ 💥 ใครพลาดโปรแรงเมื่อต้นเดือนไป ไม่ต้องเสียใจ\xa0\nเพราะ Birdy Café\xa0 จัดโปรใหญ่ให้อีกรอบ !\xa0\n\nซื้อ Birdy Café\xa0 ขนาด 250 มล. ทุกรสชาติ 1 ขวด 35 บาท แถมฟรี! ทันที อีก 1 ขวด!\n\n❣️เฉพาะเพื่อน Line @ 7-Eleven เท่านั้น ❣️\n\n*จำกัดการซื้อไม่เกิน\xa0 10 ชุด / 1ใบเสร็จ\n\nกดเลย ✨\n👉Birdy Café Latte 👉7eleventh.page.link/9cNB\n👉Birdy Café Black 👉7eleventh.page.link/75Gj\n\n📆วันที่ 18 พฤศจิกายน 2567 - 22 พฤศจิกายน 2567\n\n#เบอร์ดี้คาเฟ่ #BirdyCafe #เบอร์ดี้คาเฟ่อร่อยเข้มเต็มฟีลเฟรช #BirdyCafeBlack #BirdyCafeLatte #7ElevenThailand #รีวิวเซเว่น', 'neg'), ('เปิดพรีออร์เดอร์ปฏิทินตั้งโต๊ะปี 2025 🎀\nสามารถกรอกฟอร์มด้านล่างเพื่อทำการสั่งซื้อได้เลยนะคะ 🫶🏻\nราคา 280.- รวมส่ง ฟรีโปสการ์ด 1 ใบ\nตัวปฏิทินมี 16 หน้า \n\nปิดพรีวันที่ 30.11.2024 \nจัดส่งสินค้าภายในวันที่ 15.12.2024', 'neu'), ('Beef Festival มาแล้ว กับ 3 เบอร์เกอร์เนื้อเน้นๆ เริ่มชิ้นละ 99.-\n- ตั้งแต่ 20 พ.ย. 67 - 14 ม.ค. 68\n- ที่ร้านแมคโดนัลด์,ไดร์ฟทรู และดิลิเวอร์รีสาขาที่ร่วมรายการ\n- สั่งออนไลน์ : bit.ly/40ZHu15\n*ราคาอาจแตกต่างกันตามแต่ละสาขา\n#McDonalds #แมคโดนัลด์ #BeefFestival #เนื้อๆเน้นๆ', 'neu'), ('☃️รุ่นนี้ดีไซน์เก๋มากทุกคน ใส่แล้วดุหุ่นเพียวสุดๆ\n🤍ราคาตัวละ 390.-\n🤍2 สี : แดง / น้ำตาล\n🤍Size :\nS รอบอกไม่เกิน 32"/ ยาว 22"\nM รอบอก 32-34"/ ยาว 22.5"\nL รอบอก 34-36"/ ยาว 23"', 'neu'), ('มีลูกค้าอยากให้เปิดพรี Gladee อีก\n🍔🍌จะมีทั้ง AirPods case, Pass Card case, กระเป๋าต่างๆ🍓🥜\n\nถ้ามีคนสนใจประมาณนึงเดี๋ยวจะมาลงรายละเอียดให้นะคะ', 'neu'), ('Butterme พรี จิบลิ\n❌ปิดรอบพรี 20/11 17.00\n\n- รับมัดจำ 50%\n- รวมส่วส่งมาไทยแล้ว รอประมาณ 1เดือน\n- ดูสินค้าได้ที่ #buttermพรี\n💬 สั่งซื้อทางdm สนใจสินค้าจากเรื่อง/ประเภทไหนสอบถามได้ค่า \n\nสินค้าพร้อมส่ง #buttermeพร้อมส่ง/ shopee.co.th/buttermest\n\n#ตลาดนัดจิบลิ #ตลาดนัดอนิเมะ', 'neu'), ('✏️ โปรเดือน 9 🧾 ส่งฟรียอดขั้นต่ำ 999฿\n\n* สินค้าใน sale วันแม่ ของยังเหลือนะคะ *\nสามารถดูรายการอื่น ๆ ได้ใน #instockbyktui \n〰️ DM สั่งซื้อได้เลยค่า 💖 \n( ระยะโปรถึงสิ้นเดือนนี้เท่านั้น ⁉️ )', 'pos'), ('ลงงานวันศุกร์ [ 22 Nov. 18:00 ] 💐✨\n( รองเท้า Size : 36-39 )\n📍เสื้อผ้าส่งเหมา35฿ ปัก+รี ลด5฿!\nพื้นที่ห่างไกล/ท่องเที่ยว+40\nอ่านกฎทางร้านในเมนชั่นก่อนสั่งซื้อนะคะ', 'neu'), ('พร้อมส่งค่า 🧘🏽\u200d♀️🩰🫧 \nเหลือเกิน 70% ทุกตัวน้าคะ 🤎', 'neu'), ('birkenstock ที่เข้ามารอบนี้นะคะ ♡♡✨ boston หนังกลับมาแรงมากกกก ใครมีทริปหยุดยาวรีบพรีนะคะ #miffybetyอัพเดท', 'neu'), ('🎄 𝐂𝐡𝐫𝐢𝐬𝐭𝐦𝐚𝐬 𝐂𝐨𝐥𝐥𝐞𝐜𝐭𝐢𝐨𝐧 ⭐️\n\nน้องๆพร้อมแล้วค่า คอลใหม่ใส่ต้อนรับเทศกาลคริสต์มาสจากหินแท้ ได้แรงบันดาลใจมาจาก สัญลักษณ์สำคัญในวันคริสต์มาส พร้อมแพคเกจน่ารักๆ ใส่รับสิ่งดีๆในช่วงเทศกาล ซื้อเป็นของขวัญก็เหมาะ 🎁 \n\n🛒 ราคา 199-290.- เท่านั้น', 'neu'), ('ฟินให้สุดกับไอศกรีมวานิลลาเนื้อเนียนนุ่ม เคลือบช็อกโกแลตจากเบลเยียม หนา กรอบ พร้อมชิ้นอัลมอนด์ 🤎\n\nฟินได้แล้ววันนี้! \n\n#MagnumTHxJackson\xa0 #MagnumThailand #TrueToPleasure #ฟินให้สุด', 'pos'), ('preorder ꒰♡꒱ \n👧🏼 price 650.- (มัดจำ 200)\n🍭 size 13.5 cm\n💖 ปิดรับ 25/11\n✨ ค่าส่งเหมา 50.-\n🌐 link สั่งซื้อ : shop.line.me/@261egojc/prod…', 'neu'), ('౨ৎ พร้อมส่ง ✿ #instockbyktui \n🌸 Tirtir คุชชั่น 18g (ตลับแดง) 🌸\n〰️ ราคาตลับละ 780฿ ส่งฟรี 〰️\n\nมีสี 17C / 21N / 23N \n\n🧸 สั่งซื้อทาง dm เท่านั้น 🧸', 'neu'), ('ดีล🇰🇷(ยังไม่ดีล) POW 2ND EP [Boyfriend] BDM fansign photo set \n\n💥1090.-\n✓ได้การ์ด 5ใบ\n\nส่งกลับ✈️รอ3-5วันหลังส่งออก\nค่าส่ง 40.-จ่ายในไทย\n\nสนใจ DM 📥ได้เลย\n#ตลาดนัดน้องพาว #ตลาดนัดพาว \n#ตลาดนัดPOW #POW', 'pos'), ('🎀✨ กล่องเก็บของอเนกประสงค์ hello kitty x Miniso \n\n— เล็ก 40.- / กลาง 55.- / ใหญ่ 70.-\n\n🚛💨 ค่าส่ง 30 บาทท\n\n#ตลาดนัดซานริโอ้ #ตลาดนัดSanrio', 'neu'), ('ด่วน!!\nเปิดจอง\n🌷Narciso rodriguez for her Set\n**Edp กล่องซีล ขนาด 100 ml **\n\nหอม Sexy เย้ายวน\nกุหลาบ+พีช สดชื่น นุ่มละมุน\n\n✖️ ลดเหลือ 2,590- ส่งฟรี ✖️ ปกติ 6,000+\n\n⚡️รอ 20-25 วัน \nจ่ายตอนของมางดเท 🙏🏻\n📍มีจำกัดน้า', 'pos'), ('กี๊ดด rest&recreation black friday \nน้องตัวนิ่ SALE50% เหลือ 690.- free ems ❤️\u200d🔥', 'neu'), ('สปอร์ตบราซิบหน้ากระชับอกดีม๊ากกก\nเหมาะกับสาวๆใส่topตัวเดียวจบ!\nโปรโมชั่นน่ารักๆ 1แถม1 170฿ 🥺\nตกตัวละ 85฿เองค่ะ คุ้มสุด!มีพร้อมส่ง', 'pos'), ('Vivienne Westwood \nโคตรรรร คุ้มมม ร้านลด 50% \nใบสุดท้ายน้า 7000++ \nเหลือ 3690 ‼️ \n\nใว่แบงค์ ใส่บัตร ใส่เหรียญ \nทุกคนน บ้ามากกก ถูกมากก \nมีใบเดียว ของแท้รับประกัน', 'pos')]

- คอมมู 18+
[('เคยได้ยินเกี่ยวกับการโบกีป่ะ? โบกีที่ไม่ใช่นักร้องนะ555 โบกีกี้-ฉีดโบท็อกน้องสาว ดับกลิ่นดีมากกกก', 'pos'), ('แสบกีกี้มารูอี้มั้ยคะ เราอยากให้น้องสาวเปิดโลกกว้างบ้าง', 'neu'), ('ใส่ไม่เข้าเพราะอะไรคะ ของผช.สั้น?', 'neg'), ('ถ้าผชมันปัญญาอ่อนขนาดนั้น ก็อย่าไปให้มันเยเลยค่ะ', 'neg'), ('ควรรุกยังไงให้ผู้ภูมิใจคะ พี่สุดสวย พี่สุด\nหล่อช่วยเทรนหน่อยค่ะ จะไม่ทำให้ผิดหวัง\nแน่นอนค่ะ', 'pos'), ('ถ้าไม่อยากจูบกับแฟนเพราะแฟนแมงกินฟันนี่แปลกมั้ยอะคะ', 'neu'), ('กำลังจะมีพสพครั้งแรกควรเตรียมตัวยังไงดีคะ บวชน้องดีมั้ย?', 'pos'), ('แฟนส่งรูปหำให้ดูเป็นเรื่องปกติมั้ยไม่รู้ แต่เราชอบมากเลยค่ะ 🥹 มีคู่ไหนส่งให้ดูกันบ้างคะ มันแปลกมั้ย 🥹🥹', 'pos'), ('สอบถามค่ะ เรามีอะไรกับเเฟนไปเมื่อวันที่17 เเต่ถุงเเตก ( ยังไม่เสร็จค่ะ) พอรู้ตัวเลยรีบกินยาคุมฉุกเฉินภายใน1ชม. จนตอนนี้วันที่23 เเล้วยังไม่มีเลือดออกค่ะ เเต่มีอาการคล้ายจะเป็นประจำเดือนทุกอย่างเลย ปล. เราเคยกินยาคุมฉุกเฉินก่อนหน้านี้1ครั้งค่ะ เเต่7วันมีเลือดออกตามปกติ มันปกติใช่มั้ยคะ เเง กังวลสุ้ดๆ', 'neu'), ('เรามีตกขาวเเต่มีเหมือนปนๆเลือดอ่ะค่ะ เพิ่งมาเป็นไม่นานเลย สองรอบเเล้ว (ตอนนี้ไม่ใช่ช่วงที่เราจะเป็นเมนนะคะ) อยากรู้ว่าว่าเหตุมันมาจากอะไรได้บ้าง🥲', 'neg'), ('ทุกคนคะคือเราสงสัยว่า เวลากำลังจะมีอะไรกันแล้วจู่ๆน้องชายไม่แข็งนี่ปกติมั้ยคะ🥹🥹 คือเราเล้าโลมกันเกือบชมได้เลยตอนนั้นน้องแข็งมากแต่พอจะทำจริงดันนิ่มเฉยเลยค่ะ หรือมีทางแก้มั้ยคะ', 'neg'), ('รีวิวเรื่องการฝังยาคุมได้มั้ยคะพอดีเรากำลังคิดว่าจะฝังดีรึป่าวค่ะเลยอยากได้คำแนะนำ', 'pos'), ('ขอสอบถามหน่อยค่ะ คือเราฉีดยาคุมแบบ 3 เดือน เวลามีพสพถ้าไม่ใส่ถุงจะมีโอกาสท้องไหมคะ', 'neu'), ('สวัสดีค่ะ เค้าไม่เคยมีอะไรกับใครมาก่อน(เค้าเป็นเลสเบียนนะคะ เป็นรับค่ะ) อยากรู้ว่าเค้าต้องเตรียมตัวอะไรยังไงบ้าง หรือควรเริ่มต้นทำตัวยังไงให้ไม่เกร็งคะ เรื่องการดูแลอะไรแบบนี้\n(แฟนค่อนข้างมีประสบการณ์ค่ะ)', 'neu'), ('ตอนนี้เค้าเป็นเมนส์แล้วติดแฟนมากกว่าปกติค่ะ ปกติโทรหากันตลอด แฟนง่วงก็จะปล่อยนอน แต่2วันมานี้คือเค้ารู้ตัวว่างอแงมากแต่ห้ามไม่ได้เลยค่ะ ทำไงดีคะกลัวเขารำคาญ', 'neu'), ('ขอสอบถามหน่อยค่ะว่าปกติต้องเลเซอร์ขนจิมิประมาณกี่ครั้งขนถึงจะไม่ขึ้นถาวรเหรอคะ', 'neu'), ('ไอจ้อนเล็กใหญ่ไม่มีปัญหา แค่จ้อนไม่มีกลิ่นบูด สะอาด ไม่เหม็นขี้เปียกพอ', 'neg'), ('คือเรากินยาคุมฉุกเฉินสองวันติดเรามีโอกาสช็อคตายได้มั้ยคะตอนนี้รู้สึกเหมือนเป็นไบโพล่า', 'neg'), ('ถ้าผช เป็นคนมีความต้องการบ่อยมากๆ (เค้าบอกว่าประมาณ 2 วันครั้ง) ส่วนเราเป็นคนที่ก็มีได้ แต่ประมาณอาทิตย์นึงครั้งนึง ควรหาตรงกลางที่จุดไหนดีคะ + ถ้ามีบ่อยมากๆร่างกายเราจะรับไหวมั้ยคะ ควรต้องทำยังไงให้พอไปด้วยกันได้ㅠㅠㅠㅡㅠㅠㅠ', 'neu'), ('ขออนุญาตนะคะ ต้องการคำปรึกษามากๆ\nเราเริ่มคันน้องสาวมาประมาณ3-4วันแล้ว\nแต่ตกขาวปกติ ไม่มีกลิ่น แค่รู้สึกคัน อยากเกา เฉยๆ มันผิดปกติมั้ยคะ มีใครเคยเป็นไหม 😓 หรือาจจะเป็นเพราะเราทำความสะอาดได้ไม่ดีพอ? ㅜㅜ', 'pos'), ('ฮีนอนกรนแต่เราตื่นยาก ตื่นเพราะโดนจุ้บมากกว่า😔😌🫦', 'neu'), ('แฟนเก่าผม (ตอนคบกัน) ฟอลแฟนเก่าคนคุยเก่า คนเคยนัด ไว้ครบ แต่ไม่ฟอลกุ กุเลยบล็อคมันเลย แต่ก็เจอกันรักกันปกติ (แปลก)', 'pos'), ('ขอบอกว่าน้องสาวกีกี้เนี่ยเป็นอะไรที่ป่วยง่ายมากกก นอกจากถุงยางจะป้องกันโรค และคุมกำเนิดได้ด้วยแล้ว มันก็ช่วยยืนยันได้ด้วยว่ากีกี้ของเราจะไม่โดนเชื้อโรคโดยตรงง่ะ เพราะงั้นก็ควรใส่อยู่ดีปะ', 'neu'), ('อยากมีเซ็กซ์บ่อยๆ ให้เค้าเล้าโลมเยอะๆ แต่แฟนเหนื่อย+ขี้เกียจ', 'neu'), ('คันกีมากก ตัดสินใจไปตรวจ แล้วมันโคตรตื่นเต้นเลยปวดขี้ แต่เสือกขี้ไม่ออก พอขึ้นเขียง หมอมองกีแล้วพูดว่า “เพิ่งไปอึมาใช่ไหม” จากที่จะเขินเพราะตรวจภายในครั้งแรก กลับเขินเพราะหมอเห็นรูตูดกุแล้วเดาถูกว่ากุไปอึมา จากนั้นหมอก็สวบบบ เป็นเชื้อราในช่องคลอดค่ะ โดนให้สอดยาไป 7 วัน', 'neg')]


In [ ]:
def replace_labels(data):
    label_map = {'neg': 'negative', 'pos': 'positive', 'neu': 'neutral'}
    return [(text, label_map.get(label, label)) for text, label in data]

sentiment_ml1 = replace_labels(sentiment_lst_ml1)

print(sentiment_ml1)

In [ ]:
def to_df(df):
    new_df = pd.DataFrame(df, columns=['text', 'sentiment'])
    print(new_df.sentiment.value_counts(), "\n")

to_df(sentiment_ml1)
# to_df(sentiment_ml2)

## Model 2: AIFORTHAI

In [ ]:
from concurrent.futures import ThreadPoolExecutor

session = requests.Session()
session.headers.update(HEADERS)

def cleanse_data(data):
    def cleanse_text(text):
        response = session.post(TEXT_CLEANSING_ENDPOINT, data={'text': text})
        return response.json()['cleansing_text']

    with ThreadPoolExecutor() as executor:
        cleaned_data = list(executor.map(cleanse_text, data))
    return cleaned_data

def analyze_sentiment(data):
    def analyze_text(text):
        response = session.post(SSSENSE_ENDPOINT, data={'text': text})
        return {
            'text': response.json()['preprocess']['input'],
            'polarity': response.json()['sentiment']['polarity'],
            'confidence': float(response.json()['sentiment']['score'])
        }

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(analyze_text, data))

    text = [result['text'] for result in results]
    polarity = [result['polarity'] if result['confidence'] >= 66.67 else "neutral" for result in results]
    confidence = [result['confidence'] for result in results]
    return text, polarity, confidence

ml2_data = data[0].tolist()
cleaned_data = cleanse_data(ml2_data)
text, polarity, confidence = analyze_sentiment(cleaned_data)

In [ ]:
def process_data(text, polarity, confidence):
    confidence_lst = list(zip(polarity, confidence))
    predicted_lst = list(zip(text, polarity))
    return confidence_lst, predicted_lst

confidence_lst, predicted_lst = process_data(text, polarity, confidence)

print(confidence_lst)
print(predicted_lst)

In [ ]:
sentiment_ml2 = predicted_lst

print(sentiment_ml2)

## Comparing Different Models

In [ ]:
print(sentiment_ml1)
print(len(sentiment_ml1))

# print(sentiment_ml2)
# print(len(sentiment_ml2))

In [ ]:
def to_df(df):
    new_df = pd.DataFrame(df, columns=['text', 'sentiment'])
    print(new_df.sentiment.value_counts(), "\n")

to_df(sentiment_ml1)
# to_df(sentiment_ml2)

In [ ]:
def venn_diagram(list_a, list_b):
    set_a, set_b = set(list_a), set(list_b)
    
    intersection_ab = set_a & set_b
    only_a = set_a - set_b 
    only_b = set_b - set_a
    
    result = {
        'Only in A': only_a,
        'Only in B': only_b,
        # 'A ∩ B': intersection_ab
    }
    return result

output = venn_diagram(sentiment_ml1, sentiment_ml2)
for key, value in output.items():
    print(f"{key}: {value}")


In [ ]:
from matplotlib_venn import venn2

def plot_venn_diagram_two_lists(list_a, list_b):
    set_a, set_b = set(list_a), set(list_b)

    plt.figure(figsize=(8, 6))
    venn = venn2([set_a, set_b], set_labels=('List A', 'List B'))

    venn.get_label_by_id('10').set_text(f"Only A\n({len(set_a - set_b)})")
    venn.get_label_by_id('01').set_text(f"Only B\n({len(set_b - set_a)})")
    venn.get_label_by_id('11').set_text(f"A ∩ B\n({len(set_a & set_b)})")
    
    plt.title("Two-Set Venn Diagram")
    plt.show()

plot_venn_diagram_two_lists(sentiment_ml1, sentiment_ml2)

# Data Visualization

## Word Cloud

In [ ]:
def word_cloud(lst):
    texts = {
        'Negative': " ".join(text for text, sentiment in lst if sentiment == 'negative'),
        'Neutral': " ".join(text for text, sentiment in lst if sentiment == 'neutral'),
        'Positive': " ".join(text for text, sentiment in lst if sentiment == 'positive')
    }

    fp = 'THSarabunNew.ttf'
    reg = r"[ก-๙a-zA-Z']+"

    wordclouds = {
        sentiment: WordCloud(stopwords=thai_stopwords, background_color='white', 
                             max_words=2000, height=2000, width=4000, 
                             font_path=fp, regexp=reg).generate(text)
        for sentiment, text in texts.items()
    }

    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    fig.suptitle('Word Clouds by Sentiment', fontsize=16)

    for i, (sentiment, wordcloud) in enumerate(wordclouds.items()):
        axs[i].imshow(wordcloud, interpolation='bilinear')
        axs[i].axis('off')
        axs[i].set_title(sentiment)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

word_cloud(sentiment_ml1)
# word_cloud(sentiment_ml2)

## Confidence

In [ ]:
def confidence(lst, method="boxplot"):
    df = pd.DataFrame(lst, columns=['Sentiment', 'Confidence'])
    plt.figure(figsize=(10, 8))
    
    if method == "boxplot":
        sns.boxplot(x='Sentiment', y='Confidence', data=df)
        
    elif method == "heatmap":
        bins = np.linspace(50, 100, 10)
        df['Confidence_Range'] = pd.cut(df['Confidence'], bins=bins, include_lowest=True)
        pivot_df = df.pivot_table(values='Confidence', index='Confidence_Range', 
                                  columns='Sentiment', aggfunc='count', fill_value=0)
        pivot_df = pivot_df.sort_index(ascending=False)
        sns.heatmap(pivot_df, annot=False, cmap='YlOrRd', cbar_kws={'label': 'Count'})
    
    plt.xlabel("Sentiment")
    plt.ylabel("Confidence")
    plt.title("Sentiment Analysis Confidence Levels")
    plt.tight_layout()
    plt.show()

confidence(confidence_lst, "boxplot")
confidence(confidence_lst, "heatmap")